In [ ]:
import iris_pipeline
iris_pipeline.monkeypatch_jwst_datamodels()

import numpy as np

In [ ]:
iris_pipeline.__file__

## Test the `parse_subarray_map` function

In [ ]:
from iris_pipeline.parse_subarray_map.parse_subarray_map_step import parse_subarray_map

In [ ]:
subarray_maps_metadata = []
subarray_maps_metadata.append({"xstart":80, "ystart":70, "xsize":10, "ysize":10})  
subarray_maps_metadata.append({"xstart":10, "ystart":20, "xsize":20, "ysize":20})  

In [ ]:
def set_subarray_mask(mask_array, subarray_index, xstart, ystart, xsize, ysize):
    xstart = xstart - 1
    ystart = ystart - 1
    mask_array[ystart:ystart+ysize, xstart:xstart+xsize] =  subarray_index                 

In [ ]:
subarray_map = np.zeros((100,100), dtype=np.int16)

In [ ]:
for i, shape in enumerate(subarray_maps_metadata):
    set_subarray_mask(subarray_map, subarray_index=i+1, **shape)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(subarray_map);

In [ ]:
parse_subarray_map_output = parse_subarray_map(subarray_map)

In [ ]:
type(parse_subarray_map_output[0]["xstart"])

In [ ]:
assert subarray_maps_metadata == parse_subarray_map_output

## Test the pipeline step

In [ ]:
from iris_pipeline import ParseSubarrayMapStep

In [ ]:
im = iris_pipeline.datamodels.IRISImageModel(data=np.zeros((100,100)))

In [ ]:
im.dq[25, 25] = 16

In [ ]:
im.dq[26, 26] = 1

In [ ]:
im["subarr_map"] = subarray_map

In [ ]:
output = ParseSubarrayMapStep().run(im)

In [ ]:
for each_parsed, each_input in zip(output.meta.subarray_map, subarray_maps_metadata):
    assert each_parsed.instance == each_input

In [ ]:
# if a pixel is already flagged as subarray, don't mess it up
assert output.dq[25, 25] == 16

In [ ]:
# conserve existing flags
assert output.dq[26, 26] == 17

In [ ]:
plt.imshow(output.dq)

In [ ]:
np.testing.assert_array_equal(np.bitwise_and(output.dq, int(2**4)) > 0, subarray_map != 0)